In [ ]:
import os
import torch
import pandas as pd

from functools import partial

from utils.losses import MosLoss
from utils.model_list import xception
from utils.data_loader import get_data_loaders
from utils.configs import config
from utils.metrics import accuracy, macro_f1

from fastai.vision.all import *
from fastai.callback.tracker import SaveModelCallback
from fastai.distributed import setup_distrib, num_distrib

In [ ]:
device = torch.device('cuda:0')

In [ ]:
print(device)

In [ ]:
def get_class_map(df):
    class_map = {}
    for i in range(len(df)):
        class_map[df.loc[i, 'Species']] = df.loc[i, 'Species_Name']
    return class_map

In [ ]:
cm = get_class_map(pd.read_csv(config.DATA_CSV_PATH))
for i in range(len(cm)):
    print(i, ": ", cm[i])
print(len(cm))

In [ ]:
model_params = [config.fold]
MODEL_CKPT = os.path.abspath('model_weights/fold_{}/'.format(*model_params))

In [ ]:
net = xception(config=config)

In [ ]:
gpu = setup_distrib(config.gpu)
opt = config.optimizer
mom = config.mom
alpha = config.alpha
eps = config.eps

In [ ]:
# opt_func = partial(Adam, mom=mom, sqr_mom=alpha, eps=eps)
opt_func = partial(ranger, mom=mom, alpha=alpha, eps=eps)

In [ ]:
train_dl, valid_dl = get_data_loaders(config)

In [ ]:
dls = DataLoaders(train_dl, valid_dl)

In [ ]:
loss = MosLoss(config=config)
freeze_bn = False
save_imgs = False
train_losses = []
valid_losses = []
valid_f1s = []
lr_hist = []
metrics = [accuracy, macro_f1]

In [ ]:
learn = Learner(
    dls,
    net,
    wd=config.weight_decay,
    opt_func=opt_func,
    metrics=metrics,
    loss_func=loss,
    model_dir=MODEL_CKPT,
    path=f'model_weights/fold_{config.fold}'
)

if gpu is None:
    learn.to_parallel()
elif num_distrib() > 1:
    learn.to_distributed(gpu)

cb = SaveModelCallback(monitor='macro_f1')

In [ ]:
learn.fit_one_cycle(
    config.epochs,
    config.lr,
    div=10.0,
    pct_start=0.3,
    cbs=[cb],
)